# Pandas - Part 2
> "Python Data Science Handbook" - *Jake Vanderplas (2016)*

# Combining Datasets: Concat

Some of the most interesting studies of data come from combining different data
sources. These operations can involve anything from very straightforward concatena‐
tion of two different datasets, to more complicated database-style joins and merges
that correctly handle any overlaps between the datasets. Series and DataFrames are
built with this type of operation in mind, and Pandas includes functions and methods
that make this sort of data wrangling fast and straightforward.

Here we’ll take a look at simple concatenation of Series and DataFrames with the
pd.concat function; later we’ll dive into more sophisticated in-memory merges and
joins implemented in Pandas.

We begin with the standard imports:

```python
In[1]: import pandas as pd
 import numpy as np
```

For convenience, we’ll define this function, which creates a DataFrame of a particular
form that will be useful below:
```python
In[2]:  def make_df(cols, ind):
                """Quickly make a DataFrame"""
                data = {c: [str(c) + str(i) for i in ind] for c in cols}
                return pd.DataFrame(data, ind)
        # example DataFrame
         make_df('ABC', range(3))

Out[2]:   A  B  C
        0 A0 B0 C0
        1 A1 B1 C1
        2 A2 B2 C2
  ```
## Simple Concatenation with pd.concat
Pandas has a function, pd.concat(), which has a similar syntax to np.concatenate
but contains a number of options that we’ll discuss momentarily:
```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
 keys=None, levels=None, names=None, verify_integrity=False,
 copy=True)
 ```

`pd.concat()` can be used for a simple concatenation of Series or DataFrame objects,
just as `np.concatenate()` can be used for simple concatenations of arrays:
```python
In[6]:  ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
        ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
        pd.concat([ser1, ser2])
Out[6]: 1 A
        2 B
        3 C
        4 D
        5 E
        6 F
        dtype: object
```
It also works to concatenate higher-dimensional objects, such as DataFrames:
```python
In[7]:  df1 = make_df('AB', [1, 2])
        df2 = make_df('AB', [3, 4])
        print(df1); print(df2); print(pd.concat([df1, df2]))
        
  df1           df2             pd.concat([df1, df2])
   A  B            A  B           A B
 1 A1 B1         3 A3 B3        1 A1 B1
 2 A2 B2         4 A4 B4        2 A2 B2
 3 A3 B3        
 4 A4 B4        
```
By default, the concatenation takes place row-wise within the DataFrame (i.e.,
axis=0). Like np.concatenate, pd.concat allows specification of an axis along which
concatenation will take place. Consider the following example:
```python
In[8]:  df3 = make_df('AB', [0, 1])
        df4 = make_df('CD', [0, 1])
        print(df3); print(df4); print(pd.concat([df3, df4], axis='col'))
        
   df3            df4             pd.concat([df3, df4], axis='col')
   A  B            C  D            A  B  C  D
 0 A0 B0         0 C0 D0         0 A0 B0 C0 D0
 1 A1 B1         1 C1 D1         1 A1 B1 C1 D1
```
We could have equivalently specified axis=1; here we’ve used the more intuitive
`axis='col'`.


### Try it yourself: Create a CSV file using excel with the ff conditions:
1. Create 2 csv files with the same indices but different columns and merge using `concat` at `axis='col'`
2. Create 2 csv files with the different indices but same columns and merge using `concat` at `axis='row'`

Hint: When using colab, upload the files first to your colab environment. Copy the path and read using `pd.read_csv(<path>, index_col=0)`

In [19]:
# your code here


# Combining Datasets: Merge and Join

## Categories of Joins
The `pd.merge()` function implements a number of types of joins: the *one-to-one*,
*many-to-one*, and *many-to-many joins*. All three types of joins are accessed via an
identical call to the `pd.merge()` interface; the type of join performed depends on the
form of the input data. Here we will show simple examples of the three types of
merges, and discuss detailed options further below

### One-to-one joins

Perhaps the simplest type of merge expression is the one-to-one join, which is in
many ways very similar to the column-wise concatenation seen in “Combining Data‐
sets: Concat and Append” on page 141. As a concrete example, consider the following
two DataFrames, which contain information on several employees in a company:
```python
In[2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)
df1                     df2
 employee group         employee  hire_date
0 Bob   Accounting      0 Lisa  2004
1 Jake  Engineering     1 Bob   2008
2 Lisa  Engineering     2 Jake  2012
3 Sue   HR              3 Sue   2014
```
To combine this information into a single DataFrame, we can use the pd.merge()
function:

```python
In[3]: df3 = pd.merge(df1, df2)
 df3

Out[3]: employee   group       hire_date
        0 Bob      Accounting  2008
        1 Jake     Engineering 2012
        2 Lisa     Engineering 2004
        3 Sue      HR          2014
```

The `pd.merge()` function recognizes that each DataFrame has an **“employee”** column,
and automatically joins using this column as a key. The result of the merge is a new
DataFrame that combines the information from the two inputs. Notice that the order
of entries in each column is not necessarily maintained: in this case, the order of the
**“employee”** column differs between df1 and df2, and the `pd.merge()` function cor‐
rectly accounts for this.

### Many-to-one joins
Many-to-one joins are joins in which one of the two key columns contains duplicate
entries. For the many-to-one case, the resulting DataFrame will preserve those dupli‐
cate entries as appropriate. Consider the following example of a many-to-one join:
```python
In[4]:  df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                           'supervisor': ['Carly', 'Guido', 'Steve']})
        print(df3); print(df4); print(pd.merge(df3, df4))

df3                                  df4
  employee group      hire_date         group         supervisor
0 Bob   Accounting   2008            0 Accounting    Carly
1 Jake  Engineering  2012            1 Engineering   Guido
2 Lisa  Engineering  2004            2 HR            Steve
3 Sue   HR           2014

pd.merge(df3, df4)
 employee group         hire_date supervisor
0 Bob   Accounting      2008    Carly
1 Jake  Engineering     2012    Guido
2 Lisa  Engineering     2004    Guido
3 Sue   HR              2014    Steve
```
The resulting DataFrame has an additional column with the `“supervisor”` information,
where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins
Many-to-many joins are a bit confusing conceptually, but are nevertheless well
defined. If the key column in both the left and right array contains duplicates, then
the result is a many-to-many merge. This will be perhaps most clear with a concrete
example. Consider the following, where we have a DataFrame showing one or more
skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any
individual person:
```python
In[5]: df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                                        'Engineering', 'Engineering', 'HR', 'HR'],
                           'skills': ['math', 'spreadsheets', 'coding', 'linux',
                                        'spreadsheets', 'organization']})
print(df1); print(df5); print(pd.merge(df1, df5))
  df1                      df5
  employee group           group         skills
0 Bob   Accounting       0 Accounting    math
1 Jake  Engineering      1 Accounting    spreadsheets
2 Lisa  Engineering      2 Engineering   coding
3 Sue   HR               3 Engineering   linux
                         4 HR            spreadsheets
                         5 HR            organization

pd.merge(df1, df5)
  employee group         skills
0 Bob   Accounting      math
1 Bob   Accounting      spreadsheets
2 Jake  Engineering     coding
3 Jake  Engineering     linux
4 Lisa  Engineering     coding
5 Lisa  Engineering     linux
6 Sue   HR              spreadsheets
7 Sue   HR              organization
```


These three types of joins can be used with other Pandas tools to implement a wide
array of functionality. But in practice, datasets are rarely as clean as the one we’re
working with here. In the following section, we’ll consider some of the options pro‐
vided by pd.merge() that enable you to tune how the join operations work.


## Specification of the Merge Key
We’ve already seen the default behavior of pd.merge(): it looks for one or more
matching column names between the two inputs, and uses this as the key. However,
often the column names will not match so nicely, and pd.merge() provides a variety
of options for handling this.

### The `on` keyword
Most simply, you can explicitly specify the name of the key column using the on key‐
word, which takes a column name or a list of column names:
```python
In[6]: print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))
df1                        df2
  employee group           employee hire_date
0 Bob   Accounting       0 Lisa 2004
1 Jake  Engineering      1 Bob 2008
2 Lisa  Engineering      2 Jake 2012
3 Sue   HR               3 Sue 2014

pd.merge(df1, df2, on='employee')
 employee group         hire_date
0 Bob   Accounting      2008
1 Jake  Engineering     2012
2 Lisa  Engineering     2004
3 Sue   HR              2014
```
This option works only if both the left and right DataFrames have the specified col‐
umn name.


### The `left_on` and `right_on` keywords
At times you may wish to merge two datasets with different column names; for exam‐
ple, we may have a dataset in which the employee name is labeled as “name” rather
than “employee”. In this case, we can use the left_on and right_on keywords to
specify the two column names:
```python
In[7]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1), print(df3)
print(pd.merge(df1, df3, left_on="employee", right_on="name"))
  df1                                  df3
  employee group                       name  salary
0 Bob    Accounting                  0 Bob   70000
1 Jake   Engineering                 1 Jake  80000
2 Lisa   Engineering                 2 Lisa  120000
3 Sue    HR                          3 Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee group       name    salary
0 Bob   Accounting     Bob      70000
1 Jake  Engineering    Jake     80000
2 Lisa  Engineering    Lisa     120000
3 Sue   HR             Sue      90000
```
The result has a redundant column that we can drop if desired—for example, by
using the `drop()` method of DataFrames:
```python
In[8]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)
Out[8]: employee group         salary
        0 Bob   Accounting     70000
        1 Jake  Engineering    80000
        2 Lisa  Engineering    120000
        3 Sue   HR             90000
```


# Aggregation and Grouping
An essential piece of analysis of large data is efficient summarization: computing
aggregations like sum(), mean(), median(), min(), and max(), in which a single num‐
ber gives insight into the nature of a potentially large dataset. In this section, we’llexplore aggregations in Pandas, from simple operations akin to what we’ve seen on
NumPy arrays, to more sophisticated operations based on the concept of a groupby.
## Planets Data
Here we will use the Planets dataset, available via the Seaborn package. It gives information on planets that astronomers
have discovered around other stars (known as extrasolar planets or exoplanets for
short). It can be downloaded with a simple Seaborn command:
```python
In[2]:  import seaborn as sns
        planets = sns.load_dataset('planets')
        planets.shape
Out[2]: (1035, 6)
In[3]:  planets.head()
Out[3]:   method          number    orbital_period  mass    distance year
        0 Radial Velocity 1         269.300         7.10    77.40   2006
        1 Radial Velocity 1         874.774         2.21    56.95   2008
        2 Radial Velocity 1         763.000         2.60    19.84   2011
        3 Radial Velocity 1         326.030         19.40   110.62  2007
        4 Radial Velocity 1         516.220         10.50   119.47  2009
```
This has some details on the 1,000+ exoplanets discovered up to 2014.

## GroupBy: Split, Apply, Combine
Simple aggregations can give you a flavor of your dataset, but often we would prefer
to aggregate conditionally on some label or index: this is implemented in the so￾called groupby operation. The name “group by” comes from a command in the SQL
database language, but it is perhaps more illuminative to think of it in the terms first
coined by Hadley Wickham of Rstats fame: split, apply, combine.
### Split, apply, combine
A canonical example of this split-apply-combine operation, where the “apply” is a
summation aggregation, is illustrated in Figure 3-1.
Figure 3-1 makes clear what the GroupBy accomplishes:
* The split step involves breaking up and grouping a DataFrame depending on the
value of the specified key.
* The apply step involves computing some function, usually an aggregate, transfor‐
mation, or filtering, within the individual groups.
* The combine step merges the results of these operations into an output array.

While we could certainly do this manually using some combination of the masking,
aggregation, and merging commands covered earlier, it’s important to realize that the
intermediate splits do not need to be explicitly instantiated. Rather, the GroupBy can
(often) do this in a single pass over the data, updating the sum, mean, count, min, or
other aggregate for each group along the way. The power of the GroupBy is that it
abstracts away these steps: the user need not think about how the computation is
done under the hood, but rather thinks about the operation as a whole.

As a concrete example, we’ll start by creating the input DataFrame:
```python
In[11]: df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                           'data': range(6)}, columns=['key', 'data'])
        df
Out[11]:         key data
                0 A 0
                1 B 1
                2 C 2
                3 A 3
                4 B 4
                5 C 5
```
We can compute the most basic split-apply-combine operation with the groupby()
method of DataFrames, passing the name of the desired key column:

```python
In[12]: df.groupby('key')
Out[12]: <pandas.core.groupby.DataFrameGroupBy object at 0x117272160>
```

Notice that what is returned is not a set of DataFrames, but a DataFrameGroupBy
object. This object is where the magic is: you can think of it as a special view of the
DataFrame, which is poised to dig into the groups but does no actual computation
until the aggregation is applied. This “lazy evaluation” approach means that common
aggregates can be implemented very efficiently in a way that is almost transparent to
the user.

To produce a result, we can apply an aggregate to this DataFrameGroupBy object,
which will perform the appropriate apply/combine steps to produce the desired
result:
```python
In[13]: df.groupby('key').sum()
Out[13]: data
 key
 A 3
 B 5
 C 7
 ```
The `sum()` method is just one possibility here; you can apply virtually any common
Pandas or NumPy aggregation function, as well as virtually any valid DataFrame
operation, as we will see in the following discussion.